# Normal dataset

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import json

sys.path.append('../')
sys.path.append('../src/')
sys.path.append('../spell/')

import Reader
import ParamsExtractor3
import DataPreprocessor
import EncodeCommand
import DeepLearningAnomalyDetection2

2024-09-05 12:58:35.201770: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-05 12:58:35.251927: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 12:58:36.047176: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
fp = '../../../../../temp_logs/20240418/login01.laurel.log'

data_list = []
lines_limit = 1000 # limit reading lines, done for memory constraints
i = 0

with open(fp, 'r') as file:
    for line in file:
        i+=1
        if i > lines_limit:
            break
            # Find the index of the first '{'
        index = line.find('{')
        if index != -1:
            json_data = line[index:]
            data = json.loads(json_data)
            data_list.append(data)
                
df = pd.DataFrame(data_list)

In [3]:
df.head()

,host,ident,message,severity,facility,time
0,login01,laurel,"""1713391196.350:55343008"",""NODE"":""login01"",""SY...",info,local6,2024-04-18 00:00:03 +0200
1,login01,laurel,"""1713391198.212:55343009"",""NODE"":""login01"",""SY...",info,local6,2024-04-18 00:00:03 +0200
2,login01,laurel,"""1713391198.214:55343010"",""NODE"":""login01"",""SY...",info,local6,2024-04-18 00:00:03 +0200
3,login01,laurel,"""1713391198.220:55343011"",""NODE"":""login01"",""SY...",info,local6,2024-04-18 00:00:03 +0200
4,login01,laurel,"""1713391198.221:55343012"",""NODE"":""login01"",""SY...",info,local6,2024-04-18 00:00:03 +0200


In [4]:
df.iloc[0]['message']

'"1713391196.350:55343008","NODE":"login01","SYSCALL":{"arch":"0xc000003e","syscall":59,"success":"yes","exit":0,"items":2,"ppid":29378,"pid":37632,"auid":126858,"uid":126858,"gid":25200,"euid":126858,"suid":126858,"fsuid":126858,"egid":25200,"sgid":25200,"fsgid":25200,"tty":"pts38","ses":703112,"comm":"squeue","exe":"/opt/slurm/23.11.5/bin/squeue","key":"auditcmd","ARCH":"x86_64","SYSCALL":"execve","AUID":"cmatteuc","UID":"cmatteuc","GID":"interactive","EUID":"cmatteuc","SUID":"cmatteuc","FSUID":"cmatteuc","EGID":"interactive","SGID":"interactive","FSGID":"interactive","ARGV":["0x564021c6aff0","0x564021c6bdf0","0x564021c34470","0x8"]},"EXECVE":{"argc":3,"ARGV":["squeue","-u","cmatteuc"],"ENV":{"LD_LIBRARY_PATH":"/g100_work/PROJECTS/spack/v0.17/prod/0.17.1/install/0.17/linux-centos8-skylake_avx512/gcc-8.4.1/anaconda3-2021.05-alnfpd33vtv6tt7sf6iq5ngwv7nqgeah/lib:/cineca/prod/opt/compilers/intel/oneapi-2022/binary/mpi/latest/libfabric/lib/prov:/cineca/prod/opt/compilers/intel/oneapi-2022

In [5]:
import re
clean_log = re.sub(r'"\d+\.\d+:\d+",', '{', df.iloc[0]['message'])
clean_log += '}'
print(clean_log)

{"NODE":"login01","SYSCALL":{"arch":"0xc000003e","syscall":59,"success":"yes","exit":0,"items":2,"ppid":29378,"pid":37632,"auid":126858,"uid":126858,"gid":25200,"euid":126858,"suid":126858,"fsuid":126858,"egid":25200,"sgid":25200,"fsgid":25200,"tty":"pts38","ses":703112,"comm":"squeue","exe":"/opt/slurm/23.11.5/bin/squeue","key":"auditcmd","ARCH":"x86_64","SYSCALL":"execve","AUID":"cmatteuc","UID":"cmatteuc","GID":"interactive","EUID":"cmatteuc","SUID":"cmatteuc","FSUID":"cmatteuc","EGID":"interactive","SGID":"interactive","FSGID":"interactive","ARGV":["0x564021c6aff0","0x564021c6bdf0","0x564021c34470","0x8"]},"EXECVE":{"argc":3,"ARGV":["squeue","-u","cmatteuc"],"ENV":{"LD_LIBRARY_PATH":"/g100_work/PROJECTS/spack/v0.17/prod/0.17.1/install/0.17/linux-centos8-skylake_avx512/gcc-8.4.1/anaconda3-2021.05-alnfpd33vtv6tt7sf6iq5ngwv7nqgeah/lib:/cineca/prod/opt/compilers/intel/oneapi-2022/binary/mpi/latest/libfabric/lib/prov:/cineca/prod/opt/compilers/intel/oneapi-2022/binary/mpi/latest/libfabr

In [6]:
data = json.loads(clean_log)

In [7]:
def inspect_log_error(log: str, error_position: int, radius=20):
    start = max(0, error_position - radius)
    end = min(len(log), error_position + radius)
    return log[start:end]

log = clean_log  # The problematic log
error_position = 1960  # As indicated by the error message

print(inspect_log_error(log, error_position, 2))


}}}


In [8]:
import json
import re

def parse_audit_log(log):
    # Initialize an empty dictionary to hold the key-value pairs
    result = {}

    # Placeholder for timedelta, pid timedelta, and anomaly-related logic
    result['id_anomalies'] = 0  # Placeholder, implement anomaly logic
    result['num_id_anomalies'] = 0  # Placeholder, implement anomaly counting
    
    # Clean the log string by removing unnecessary quotes at the beginning and end
    # and replacing any non-standard characters like '\x1d'
    timestamp_match = re.match(r'"(\d+\.\d+:\d+)",', log)
    if timestamp_match:
        # Extract and store the timestamp in the result dictionary
        result['unix_time'] = timestamp_match.group(1)
    else:
        result['unix_time'] = None
        
    clean_log = re.sub(r'"\d+\.\d+:\d+",', '{', log)
    clean_log += '}'
    #clean_log = log.strip('"')
    #print(clean_log)
    #print('===========================0')

    try:
        # Convert the cleaned JSON log to a dictionary
        data = json.loads(clean_log)

        # Extract relevant fields
        syscall = data.get('SYSCALL', {})
        result['cwd'] = data.get('CWD', {}).get('cwd')
        result['exe'] = syscall.get('exe')
        result['exit'] = syscall.get('exit')
        result['items'] = syscall.get('items')
        result['ppid'] = syscall.get('ppid')
        result['pid'] = syscall.get('pid')
        result['comm'] = syscall.get('comm')

        for str_id in ['auid', 'uid', 'gid', 'euid', 'suid', 'fsuid', 'egid', 'sgid', 'fsgid', 'AUID', 'UID', 'GID', 'EUID', 'SUID', 'FSUID', 'EGID', 'SGID', 'FSGID']:
            result[str_id] = syscall.get(str_id)

    except json.JSONDecodeError as e:
        pass
        #print(f"Error: Couldn't parse the JSON part of the log. {e}")
    
    return result

# Example usage
log = parse_audit_log(df.iloc[0]['message'])
print(log)


{'id_anomalies': 0, 'num_id_anomalies': 0, 'unix_time': '1713391196.350:55343008', 'cwd': '/g100/home/userexternal/cmatteuc/Nek5000/run/full_circle_06', 'exe': '/opt/slurm/23.11.5/bin/squeue', 'exit': 0, 'items': 2, 'ppid': 29378, 'pid': 37632, 'comm': 'squeue', 'auid': 126858, 'uid': 126858, 'gid': 25200, 'euid': 126858, 'suid': 126858, 'fsuid': 126858, 'egid': 25200, 'sgid': 25200, 'fsgid': 25200, 'AUID': 'cmatteuc', 'UID': 'cmatteuc', 'GID': 'interactive', 'EUID': 'cmatteuc', 'SUID': 'cmatteuc', 'FSUID': 'cmatteuc', 'EGID': 'interactive', 'SGID': 'interactive', 'FSGID': 'interactive'}


In [9]:
# Apply the parse_audit_log function to each row in the 'log' column
df_expanded = df['message'].apply(parse_audit_log).apply(pd.Series)

# Concatenate the new columns with the original DataFrame
df = pd.concat([df, df_expanded], axis=1)
df.head()

,host,ident,message,severity,facility,time,id_anomalies,num_id_anomalies,unix_time,cwd,...,fsgid,AUID,UID,GID,EUID,SUID,FSUID,EGID,SGID,FSGID
0,login01,laurel,"""1713391196.350:55343008"",""NODE"":""login01"",""SY...",info,local6,2024-04-18 00:00:03 +0200,0,0,1713391196.350:55343008,/g100/home/userexternal/cmatteuc/Nek5000/run/f...,...,25200.0,cmatteuc,cmatteuc,interactive,cmatteuc,cmatteuc,cmatteuc,interactive,interactive,interactive
1,login01,laurel,"""1713391198.212:55343009"",""NODE"":""login01"",""SY...",info,local6,2024-04-18 00:00:03 +0200,0,0,1713391198.212:55343009,/var/lib/pacemaker/cores,...,0.0,unset,root,root,root,root,root,root,root,root
2,login01,laurel,"""1713391198.214:55343010"",""NODE"":""login01"",""SY...",info,local6,2024-04-18 00:00:03 +0200,0,0,1713391198.214:55343010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,login01,laurel,"""1713391198.220:55343011"",""NODE"":""login01"",""SY...",info,local6,2024-04-18 00:00:03 +0200,0,0,1713391198.220:55343011,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,login01,laurel,"""1713391198.221:55343012"",""NODE"":""login01"",""SY...",info,local6,2024-04-18 00:00:03 +0200,0,0,1713391198.221:55343012,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def read(fp):
    data_list = []
    lines_limit = 100000 # limit reading lines, done for memory constraints
    i = 0
    
    with open(fp, 'r') as file:
        for line in file:
            i+=1
            if i > lines_limit:
                break
                # Find the index of the first '{'
            index = line.find('{')
            if index != -1:
                json_data = line[index:]
                data = json.loads(json_data)
                data_list.append(data)
                    
    df = pd.DataFrame(data_list)
    return df

In [11]:
def normalize(df, col):
    if col not in df.columns:
        print(f"Error, column {col} not in df")
        return None
    df_expanded = pd.json_normalize(df[col])
    #print(df_expanded.head())
    df = df.drop(col, axis=1)
    df = df.join(df_expanded, rsuffix=f'_{col}')
    return df

In [12]:
# Final Preprocessing Function
def preprocess_laurel(df):

    columns_to_keep = ['cwd', 'exit', 'items', 'ppid', 'pid', 'comm', 'timedelta', 'pid_timedelta', 'ppid_timedelta', 'id_anomalies', 'num_id_anomalies', 'ARGV_PROCTITLE_str']
    columns_to_drop = ['unix_time', 'time', 'pid_time', 'ppid_time']
    
    try:
        df['ARGV_PROCTITLE_str'] = df['ARGV_PROCTITLE'].apply(lambda x: ' '.join(x))
    except:
        columns_to_keep.remove('ARGV_PROCTITLE_str')
        pass

    try:
        time_serie = df['unix_time']
        df['unix_time'] = pd.to_numeric(time_serie.str.split(':').str[0])
        df['timedelta'] = df['unix_time'].diff()
        df['time'] = pd.to_datetime(df['unix_time'], unit='s')
    except:
        pass

    try:
        df['ppid_time'] = pd.to_datetime(pd.to_numeric(df['PPID.EVENT_ID'].str.split(':').str[0]), unit='s')
        df['ppid_timedelta'] = df['ppid_time'].diff()
        columns_to_keep.remove('ppid_timedelta')
        columns_to_drop.remove('ppid_time')
    except:
        columns_to_keep.remove('ppid_timedelta')
        columns_to_drop.remove('ppid_time')
        pass

    try:
        df['pid_time'] = pd.to_datetime(pd.to_numeric(df['PID.EVENT_ID'].str.split(':').str[0]), unit='s')
        df['pid_timedelta'] = df['pid_time'].diff()
        columns_to_keep.remove('pid_timedelta')
        columns_to_drop.remove('pid_time')
    except:
        columns_to_keep.remove('pid_timedelta')
        columns_to_drop.remove('pid_time')
        pass
    
    df = df.drop(columns_to_drop, axis=1)

    columns = ['auid', 'uid', 'gid', 'euid', 'suid', 'fsuid', 'egid', 'sgid', 'fsgid']
    
    def check_row_id_num(row):
        return pd.Series({col: row[col] == 0 for col in columns})
    
    checks = df.apply(check_row_id_num, axis=1)
    df['num_id_anomalies'] = checks.sum(axis=1)
    df = df.drop(columns, axis=1)

    columns = ['AUID', 'UID', 'GID', 'EUID', 'SUID', 'FSUID', 'EGID', 'SGID', 'FSGID']
    
    # Use apply to efficiently check values for each row
    def check_id_row(row):
        try:
            return pd.Series({col: int(not(row[col] in row['UID_GROUPS'])) for col in columns})
        except:
            return None
    
    checks = df.apply(check_id_row, axis=1)
    
    df['id_anomalies'] = checks.sum()
    df = df.drop(columns, axis=1)
    try:
        df = df.drop('UID_GROUPS', axis=1)
    except:
        pass

    # filter final columns
    df = df[columns_to_keep]

    return df

In [13]:
df = preprocess_laurel(df)

In [14]:
def save(df, save_fp):
    # Save this df to file
    #save_fp = '../data/laurel_anomalous_new/save1.csv'
    df.to_csv(save_fp, index=False)

In [ ]:
df = read(fp)
df_expanded = df['message'].apply(parse_audit_log).apply(pd.Series)
df = pd.concat([df, df_expanded], axis=1)
df.head()
df = preprocess_laurel(df)
save(df, fp + '.csv')

In [ ]:
normal_dataset_fp = fp + '.csv'
new_path = '../cleaned_data/normal.csv'
save(df, new_path)